## Parker Data Prep and Analysis
- of parking types
- of parking garages (share of parking types)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import holidays

# ML/Clustering stuff
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# CONFIG

DATA_PATH = "/Volumes/karsten.schroer@uni-koeln.de/Uni/Research/04_CASE_Clustered_EV_Charging/"
OUT_PATH = ""
HOLIDAY_YEARS = [2019, 2020]

#### Data Preparation

In [3]:
# Load Data
parkings = pd.read_csv(DATA_PATH+"Parking_Data/Parking_Data_Combined.csv")
parkings = parkings[["EntryDateTime","ExitDateTime","HoursStay","MinutesStay","RevenueAmount","SiteID","Year"]]

# limit parking duration to 48h (let's not do this for now!)
parkings = parkings[parkings["MinutesStay"]<48*60]

# some info
print("Num facilities: {}".format(len(parkings["SiteID"].unique())))
print("Num parking events: {}".format(len(parkings)))

Num facilities: 7
Num parking events: 3843007


In [4]:
holidays_ger = []

# Extract Holidays
for item in holidays.Germany(years=HOLIDAY_YEARS).items():
    holidays_ger.append(item[0])

# Add Christmas Eve and New Years Eve manually since it is not an official holiday
holidays_ger.append(datetime(year=2019, month=12, day=24).date())
holidays_ger.append(datetime(year=2019, month=12, day=31).date())


def weekday_flag (x):
    if x < 4:
        return 1
    else:
        return 0

def holiday_flag (x):
    
    x = pd.to_datetime(x).date()
    
    if x in holidays_ger:
        return 1
    else:
        return 0
    
def minutes_from_midnight (datetime_obj):
    mfm = datetime_obj.hour * 60 + datetime_obj.minute
    
    return mfm

In [5]:
# Create required features

#get temporal features
parkings["EntryDateTime"] = pd.to_datetime(parkings["EntryDateTime"])
parkings["ExitDateTime"] = pd.to_datetime(parkings["ExitDateTime"])

parkings['EntryMFM'] = parkings['EntryDateTime'].apply (lambda x: minutes_from_midnight(x) )
parkings['ExitMFM'] = parkings['ExitDateTime'].apply (lambda x: minutes_from_midnight(x) )

parkings["EntryDate"] = parkings["EntryDateTime"].apply(lambda x: x.date())
parkings["ExitDate"] = parkings["ExitDateTime"].apply(lambda x: x.date())
parkings["EntryHour"] = parkings["EntryDateTime"].apply(lambda x: x.hour)
parkings["ExitHour"] = parkings["ExitDateTime"].apply(lambda x: x.hour)

# time is circular; we can reperesent by sine and cosine tranformation (see: https://datascience.stackexchange.com/questions/8799/boundary-conditions-for-clustering)
parkings["EntryHour_sin"] = parkings["EntryHour"].apply(lambda t: np.sin(2*np.pi*(t/24)))
parkings["EntryHour_cos"] = parkings["EntryHour"].apply(lambda t: np.cos(2*np.pi*(t/24)))
parkings["EntryMin_sin"] = parkings["EntryMFM"].apply(lambda t: np.sin(2*np.pi*(t/(24*60))))
parkings["EntryMin_cos"] = parkings["EntryMFM"].apply(lambda t: np.cos(2*np.pi*(t/(24*60))))

parkings["ExitHour_sin"] = parkings["ExitHour"].apply(lambda t: np.sin(2*np.pi*(t/24)))
parkings["ExitHour_cos"] = parkings["ExitHour"].apply(lambda t: np.cos(2*np.pi*(t/24)))
parkings["ExitMin_sin"] = parkings["ExitMFM"].apply(lambda t: np.sin(2*np.pi*(t/(24*60))))
parkings["ExitMin_cos"] = parkings["ExitMFM"].apply(lambda t: np.cos(2*np.pi*(t/(24*60))))

# Entry day
parkings["EntryDayOfWeek"] = parkings["EntryDateTime"].apply(lambda x: x.weekday())    
parkings["EntryWeekday_yn"] = parkings["EntryDayOfWeek"].apply(lambda x: weekday_flag(x))
parkings["EntryHoliday_yn"] = parkings["EntryDateTime"].apply(lambda x: holiday_flag(x)) # axis = 1 for row-wise on entire data frame

In [6]:
parkings.to_csv(DATA_PATH+"Parking_Data/Parking_Data_Combined_CLEAN.csv")

### Facility-level Analyses

Unique Parkers per day

In [7]:
# data
parkings = pd.read_csv(DATA_PATH+"Parking_Data/Parking_Data_Combined_CLEAN.csv")
parkings["EntryDateTime"] = pd.to_datetime(parkings["EntryDateTime"])
parkings["EntryDate"] = parkings["EntryDateTime"].apply(lambda x: x.replace(second=0, minute=0, hour=0))

In [8]:
parkings_daily = parkings.groupby(["SiteID","EntryDate"],as_index=False).count()
unique_vehicles_daily = parkings_daily.groupby("SiteID",as_index=False).agg({"HoursStay":"min","EntryDateTime":"max","ExitDateTime":"mean"})

unique_vehicles_daily.columns=["SiteID","MinUniqueVehicles","MaxUniqueVehicles","MeanUniqueVehicles"]
unique_vehicles_daily["MeanUniqueVehicles"] = unique_vehicles_daily["MeanUniqueVehicles"].apply(lambda x: round(x))

In [9]:
unique_vehicles_daily

,SiteID,MinUniqueVehicles,MaxUniqueVehicles,MeanUniqueVehicles
0,Facility_1,122,4813,1877
1,Facility_2,51,717,468
2,Facility_3,565,3359,2272
3,Facility_4,74,8492,3094
4,Facility_5,38,2705,1171
5,Facility_6,16,2073,880
6,Facility_KoeBogen,8,2115,1002


Total number of parking spots (assumed to be equal to max occupancy)

In [10]:
def round_down_to_base (num,base):
    return num - (num%base)  

In [11]:
# settings
facilities = ["Facility_1","Facility_2","Facility_3","Facility_4","Facility_5","Facility_6","Facility_KoeBogen"]#["Facility_1","Facility_2","Facility_3","Facility_4","Facility_5","Facility_6","Facility_KoeBogen"]
agg_level = 5 #minutes

# set entry/exit time to five min res
parkings["EntryDateTime"] = pd.to_datetime(parkings["EntryDateTime"])
parkings["EntryDateTime{}min".format(agg_level)] = parkings["EntryDateTime"].apply(lambda x: x.replace(second=0, minute=round_down_to_base(x.minute,agg_level)))

parkings["ExitDateTime"] = pd.to_datetime(parkings["ExitDateTime"])
parkings["ExitDateTime{}min".format(agg_level)] = parkings["ExitDateTime"].apply(lambda x: x.replace(second=0, minute=round_down_to_base(x.minute,agg_level)))#.apply(lambda x: x.replace(second=0, microsecond=0))

# count entries and exits per time period in separate df
entry_counts = parkings[["SiteID","EntryDateTime{}min".format(agg_level),"ExitDateTime{}min".format(agg_level)]].groupby(["SiteID","EntryDateTime{}min".format(agg_level)]).count()
entry_counts.columns=["EntryCounts"]
entry_counts.reset_index(inplace=True)

exit_counts = parkings[["SiteID","EntryDateTime{}min".format(agg_level),"ExitDateTime{}min".format(agg_level)]].groupby(["SiteID","ExitDateTime{}min".format(agg_level)]).count()
exit_counts.columns=["ExitCounts"]
exit_counts.reset_index(inplace=True)

In [12]:
# compute target index
target_index = pd.DataFrame()
for facility in facilities:
    df=pd.DataFrame()
    df["datetime"]= pd.date_range(start=parkings["EntryDateTime{}min".format(agg_level)].min(), end=parkings["ExitDateTime{}min".format(agg_level)].max(),freq="{}S".format(agg_level*60))
    df["facility"] = facility
    df["key"] = df.apply(lambda x: "{}_{}".format(x.facility,x.datetime),axis=1) 

    #print(facility)

    target_index = target_index.append(df) 
        

# merge entries and exits

# create keys
entry_counts["key"] = entry_counts.apply(lambda x: "{}_{}".format(x.SiteID,x.EntryDateTime5min),axis=1) 
exit_counts["key"] = exit_counts.apply(lambda x: "{}_{}".format(x.SiteID,x.ExitDateTime5min),axis=1) 

# merge with target
occupancy = target_index.merge(entry_counts[["key","EntryCounts"]], on="key", how="left")
occupancy = occupancy.merge(exit_counts[["key","ExitCounts"]], on="key", how="left")
occupancy.fillna(0,inplace=True)

In [13]:
# get net occupancy change per faclity, cluster and period
occupancy["NetOccupancyChange"] = occupancy["EntryCounts"]-occupancy["ExitCounts"]

# get total occupancy as cumulative sum

occupancy_final = pd.DataFrame()

for facility in occupancy["facility"].unique():
        
    df = occupancy[(occupancy["facility"]==facility)]
    df.sort_values(by="datetime",inplace=True)
    df["TotalOccupancy"] = df["NetOccupancyChange"].cumsum()
    occupancy_final = occupancy_final.append(df)
        
occupancy_final["date"] = occupancy_final["datetime"].apply(lambda x: x.date())

In [14]:
num_parking_spots = pd.DataFrame(occupancy_final.groupby("facility").max()["TotalOccupancy"])
num_parking_spots.reset_index(inplace=True)
num_parking_spots.columns=["SiteID","NumParkingSpots"]

In [15]:
num_parking_spots

,SiteID,NumParkingSpots
0,Facility_1,1271.0
1,Facility_2,276.0
2,Facility_3,1226.0
3,Facility_4,2167.0
4,Facility_5,715.0
5,Facility_6,826.0
6,Facility_KoeBogen,508.0


In [16]:
num_parking_spots.to_csv(DATA_PATH+"Parking_Data/parking_size.csv")